# Análise dos produtos comercializados

## Criar directoria de apoio 

In [11]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('downloads').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [12]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = APIKEYHERE
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias
* Copia a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


In [13]:
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    APIKEY = config['comtrade']['key']

comtrade.init(APIKEY)

## Parâmetros gerais que não mudam



In [14]:
m49_angola = 24
m49_brazil = 76
m49_cabo_verde = 132
m49_china = 156
m49_hong_kong = 344
m49_macau = 446
m49_guine_equatorial = 226
m49_guine_bissau = 624
m49_mozambique = 508
m49_portugal = 620
m49_stome_principe = 678
m49_timor = 626

# make list of Portuguese Speaking Countries
m49_plp = [m49_angola,m49_brazil,m49_cabo_verde,m49_guine_bissau,
            m49_guine_equatorial,m49_mozambique,m49_portugal,
            m49_stome_principe,m49_timor]
m49_plp_list = ",".join(map(str,m49_plp))




## China, categorias de produtos mais importantes nas trocas com os PLP

Obtem os totais agregados com nível 2 de código HS e lista os primeiros

In [15]:
import comtrade
import pandas as pd


rank_filter = 5  # número de importações mais relevantes
years = "2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021"
flowCode = 'M,X'
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
            'flowCode','primaryValue']
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode="AG2",
                     reporterCode=m49_china,
                     partnerCode=m49_plp_list,
                     period=years,
                     timeout=120
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear','flowCode'])["primaryValue"].rank(method="dense", ascending=False)
# convert rank column to int
pco['rank'] = pco['rank'].astype(int)

pco_top5 = pco[pco['rank'] <= rank_filter]
cmdCodes_top5 = pco_top5['cmdCode'].unique()
print(f"Categorias de produtos envolvidas: {cmdCodes_top5}")
# save to Excel
pco_top5_sorted = pco_top5[pco_cols].set_index(['reporterDesc','partnerDesc','refYear','flowCode','rank']).sort_index()

# Prepare file name for Excel output
filename_note=f"{years.replace(',','_')}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./downloads/china_plp_top5_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)
# Save data
pco_top5_sorted.to_excel(excel_file, sheet_name="comtrade", startrow=2)
excel_file.close()
# Write title in first row TBD
excel_file_title = f"Trocas Comerciais China-PLP, top {rank_filter} produtos (M=Importações, X=Exportações), {years}, valores USD"
print(excel_file_title)
print("Guardado em:",excel_file_name)

# show
pd.options.display.max_colwidth=100
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 100
pco_top5_sorted

Categorias de produtos envolvidas: ['27' '64' '07' '85' '62' '34' '44' '73' '39' '63' '03' '25' '88' '52'
 '16' '83' '87' '84' '71' '20' '90' '40' '72' '74' '76' '94' '68' '22'
 '01' '26' '12' '47' '29' '24' '15' '54' '41' '17' '02' '89' '61' '82'
 '95' '55' '10' '70' '60' '09' '69' '99' '92' '49' '38' '30' '58' '48'
 '65' '97' '33' '56' '08' '04' '21' '96' '28' '31' '86' '42' '45' '91'
 '32' '06' '14' '78']
Trocas Comerciais China-PLP, top 5 produtos (M=Importações, X=Exportações), 2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021, valores USD
Guardado em: ./downloads/china_plp_top5_2000_2001_2002_2003_2004_2005_2006_2007_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_2018_2019_2020_2021_M_X.xlsx


cmdCode  \
reporterDesc partnerDesc refYear flowCode rank           
China        Angola      2000    M        1         27   
                                          2         44   
                                          3         73   
                                          4         39   
                                          5         85   
...                                                ...   
             Timor-Leste 2021    X        1         73   
                                          2         72   
                                          3         39   
                                          4         85   
                                          5         27   

                                                                                                                                            cmdDesc  \
reporterDesc partnerDesc refYear flowCode rank                                                                                                        
China        Angola      2000    M        1     Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral w...   
                                          2                                                                Wood and articles of wood; wood charcoal   
                                          3                                                                                  Iron or steel articles   
                                          4                                                                           Plastics and articles thereof   
                                          5     Electrical machinery and equipment and parts thereof; sound recorders and reproducers; televisio...   
...                                                                                                                                             ...   
             Timor-Leste 2021    X        1                                                                                  Iron or steel articles   
                                          2                                                                                          Iron and steel   
                                          3                                                                           Plastics and articles thereof   
                                          4     Electrical machinery and equipment and parts thereof; sound recorders and reproducers; televisio...   
                                          5     Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral w...   

                                                   primaryValue  
reporterDesc partnerDesc refYear flowCode rank                   
China        Angola      2000    M        1    1,842,016,530.00  
                                          2          671,664.00  
                                          3            4,055.00  
                                          4              600.00  
                                          5               82.00  
...                                                         ...  
             Timor-Leste 2021    X        1       46,230,023.00  
                                          2       22,874,181.00  
                                          3       17,001,590.00  
                                          4       16,813,847.00  
                                          5       16,094,958.00  

[1733 rows x 3 columns]

## China, detalhe das exportações mais importantes dos PLP para a China

Em dois passos: 
* obtém as categorias de nível 2 mais importantes de cada país
* pesquisa todos as subcategorias de cada

In [111]:

import ipywidgets as widgets
from IPython.display import display

rank_filter = 5  # número de importações mais relevantes
years = "2021"
partnerCode = m49_angola # 
flowCode="M"

# select year with widget from list range(2000,2022)

years = widgets.SelectMultiple(
    options=range(2000,2022),
    value=[2021],
    description='Ano:',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
partnerCodeWidget = widgets.Dropdown(
    options=[("Angola", m49_angola), ("Brazil", m49_brazil), ("Cabo Verde", m49_cabo_verde), ("Guiné Bissau", m49_guine_bissau), ("Guiné Equatorial", m49_guine_equatorial), ("Mozambique", m49_mozambique), ("Portugal", m49_portugal), ("São Tomé e Príncipe", m49_stome_principe), ("Timor-Leste", m49_timor)],
    # value=[("Angola",m49_angola)],
    description='País:',
    disabled=False,
)

# select flowCode widget from list M=Importações, X=Exportações
flowCodeWidget = widgets.Dropdown(
    options=[("Importações", "M"), ("Exportações", "X")],
    value="M",
    description='Tipo:',
    disabled=False, 
)



In [112]:

display(years)
display(partnerCodeWidget)
display(flowCodeWidget)


SelectMultiple(description='Ano:', index=(21,), options=(2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,…

Dropdown(description='País:', options=(('Angola', 24), ('Brazil', 76), ('Cabo Verde', 132), ('Guiné Bissau', 6…

Dropdown(description='Tipo:', options=(('Importações', 'M'), ('Exportações', 'X')), value='M')

In [115]:
import comtrade

yearsList = ",".join(list(map(str,years.value)))
partnerCode = partnerCodeWidget.value


# decode country
country_name = comtrade.COUNTRY_CODES.get(partnerCode)

flow = flowCodeWidget.value
print(yearsList,country_name,flow)

2021 Brazil M


In [116]:
import comtrade
from comtrade import COUNTRY_CODES, HS_CODES, HS_CODES_DF, HS_CODES_L2_DF
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']

pco_cols_detail = ['reporterDesc','partnerDesc','refYear','cmdCode','cmdDesc',
            'flowCode','primaryValue', 'isAggregate']

# first we collect the top commodity level 2 codes            
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     cmdCode="AG2",
                     reporterCode=m49_china,
                     partnerCode=partnerCode,
                     period=yearsList,
                     timeout=30,
                     echo_url=True
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
# get the countries
countries = pco_top5.partnerDesc.unique()
country_cmd_top5_codes = dict()

# get the detailed commodity codes for the top of each country
for country in countries:
    l2_codes = pco_top5[pco_top5.partnerDesc == country]['cmdCode'].unique()
    print(country,l2_codes)
    hs_details = []
    for l2_code in l2_codes:
        l2_sub_codes = list(HS_CODES_DF[HS_CODES_DF.hscode.str.startswith(l2_code)]['hscode'])
        hs_details = hs_details + l2_sub_codes
    # print(hs_details)
    country_cmd_top5_codes[country] = hs_details.copy()

# now we fetch the detail
countryDesc = COUNTRY_CODES[partnerCode]
country_cmd_codes = ",".join(country_cmd_top5_codes[countryDesc])
print(country_cmd_codes)

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     cmdCode=country_cmd_codes, # sometimes the detail commodity codes generates errors
                     reporterCode=m49_china,
                     partnerCode=partnerCode,
                     partner2Code=0,
                     period=yearsList,
                     echo_url=True,
                     timeout=60
                     )
df[pco_cols_detail].sort_values('primaryValue',ascending=False).to_excel(f"./downloads/china_plp_{years}_{flowCode}_detail_{countryDesc}.xlsx")
print("Mostrando o top 100 geral")
df[df.isAggregate == False][pco_cols_detail].sort_values('primaryValue',ascending=False).head(100)

https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=2021&partnerCode=76&partner2Code=0&cmdCode=AG2&flowCode=M&customsCode=C00&subscription-key=HIDDEN
Brazil ['26' '12' '27' '02' '47']
26,2601,260111,260112,260120,2602,260200,2603,260300,2604,260400,2605,260500,2606,260600,2607,260700,2608,260800,2609,260900,2610,261000,2611,261100,2612,261210,261220,2613,261310,261390,2614,261400,2615,261510,261590,2616,261610,261690,2617,261710,261790,2618,261800,2619,261900,2620,262011,262019,262021,262029,262030,262040,262060,262091,262099,2621,262110,262190,12,1201,120110,120190,1202,120230,120241,120242,1203,120300,1204,120400,1205,120510,120590,1206,120600,1207,120710,120721,120729,120730,120740,120750,120760,120770,120791,120799,1208,120810,120890,1209,120910,120921,120922,120923,120924,120925,120929,120930,120991,120999,1210,121010,121020,1211,121120,121130,121140,121150,121160,121190,1212,121221,121229,121291,121292,121293,121294,121299,1213,121300,1214,121410,121490,27,2701

,reporterDesc,partnerDesc,refYear,cmdCode,cmdDesc,flowCode,primaryValue,isAggregate
35,China,Brazil,2021,260111,Iron ores and concentrates; non-agglomerated,M,"38,608,822,663.00",False
18,China,Brazil,2021,120190,"Soya beans; other than seed, whether or not broken",M,"33,120,436,899.00",False
66,China,Brazil,2021,270900,"Oils; petroleum oils and oils obtained from bituminous minerals, crude",M,"15,355,791,271.00",False
2,China,Brazil,2021,020230,"Meat; of bovine animals, boneless cuts, frozen",M,"4,620,523,121.00",False
80,China,Brazil,2021,470329,"Wood pulp; chemical wood pulp, soda or sulphate, (other than dissolving grades), semi-bleached o...",M,"3,880,422,880.00",False
5,China,Brazil,2021,020329,"Meat; of swine, n.e.c. in item no. 0203.2, frozen",M,"1,617,075,329.00",False
12,China,Brazil,2021,020714,"Meat and edible offal; of fowls of the species Gallus domesticus, cuts and offal, frozen",M,"1,455,270,569.00",False
40,China,Brazil,2021,260300,Copper ores and concentrates,M,"712,835,975.00",False
36,China,Brazil,2021,260112,Iron ores and concentrates; agglomerated (excluding roasted iron pyrites),M,"512,203,129.00",False
77,China,Brazil,2021,470200,"Wood pulp; chemical wood pulp, dissolving grades",M,"403,994,727.00",False


## Países de consignação

### Partner2

O `partner2` é uma novidade dos dados comtrade. Procura registar o país de "consignação".

No caso das importações é o país que despachou os bens para o país que importa,
sem que tenha ocorrido entre o país de origem (_partner_) nenhuma transação
que modifique o estatuto legal (denominação de origem?) dos bens:

> The country of consignment in the case of imports is the country from which goods
 were dispatched to the importing country, without any commercial transactions 
 or other operations that change the legal status of the goods taking 
 place in any intermediate country.


#### Resultados quand o partner2Code não é especificado na pesquisa



Se a pesquisa não especifica partner2Code, alguns anos produzem mais do que uma linha por par _reporter/partner_  
com diferentes valores. Por exemplo, se a China for o `reporter` e a Guiné Equatorial o `partner` nos anos 2015, 2016, 2017 aparece::
* Uma linha por `partner2Code`, incluindo uma linha em que o `partner2` é igual ao `partner` (importações diretas).
* Uma linha adicional com `partner2Code` igual a zero que contém o total agregado das outras linhas com `partner2Code` explícito.
* Isso significa que existe duplicação do total.
  
|    | reporterDesc   | partnerDesc       |   partner2Code | partner2Desc         |   refYear | cmdCode   | flowCode   | primaryValueFormated   |
|---:|:---------------|:------------------|---------------:|:---------------------|----------:|:----------|:-----------|:-----------------------|
|  3 | China          | Equatorial Guinea |            344 | China, Hong Kong SAR |      2015 | TOTAL     | M          | 59.0                   |
|  1 | China          | Equatorial Guinea |             56 | Belgium              |      2015 | TOTAL     | M          | 2,435.0                |
|  2 | China          | Equatorial Guinea |            226 | Equatorial Guinea    |      2015 | TOTAL     | M          | 1,166,493,970.0        |
|  0 | China          | Equatorial Guinea |              0 | nan                  |      2015 | TOTAL     | M          | 1,166,496,464.0        |


Para evitar isso tem de se chamar a API com partner2Code = 0, para que os resultados de 2015,2016,2017 excluam
a decomposição. Se partner2Code=None as linhas adicionais aparecem.


Exemplo de resultados se o `partner2Code` for None.

Alterar as variáveis seguintes para testar.


In [31]:
period = "2015" ## if freqCode M  use aaaamm
flow = "M"
cmdCode = 'TOTAL'
reporterCode = m49_china
partnerCode = m49_guine_equatorial
# None: total and subtotal per country
# 0: only total
# > 0: only this country code
partner2Code = None

Exemplo como com `partner2Code = None` aparecem linhas
com detalhe de partner linhas de total

In [32]:
import comtrade

partner2Code = None

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format

cols_partner2 = ['reporterDesc','partnerDesc','partner2Code','partner2Desc','refYear',
        'flowDesc','primaryValue','cmdCode','qty','qtyUnitCode','qtyUnitAbbr','customsCode']
cols_partner2_qty_weight = ['reporterDesc','partnerDesc','partner2Code','partner2Desc','refYear','cmdDesc',
        'flowCode','primaryValue',
        'qty','qtyUnitCode','qty','isQtyEstimated','altQtyUnitCode', 'altQtyUnitAbbr', 'altQty', 'isAltQtyEstimated',
       'netWgt', 'isNetWgtEstimated', 'grossWgt', 'isGrossWgtEstimated',
        'motCode']

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     partner2Code=partner2Code,
                     cmdCode=cmdCode,
                     customsCode=None,
                     qtyUnitCodeFilter=-1,
                     period=period,
                     timeout=30, echo_url=True
                     )
result = df.sort_values(['partner2Code','primaryValue'], ascending=[True,False])[cols_partner2]
# print(result.to_markdown())
result

https://comtradeapi.un.org/public/v1/preview//C/A/HS?reporterCode=156&period=2015&partnerCode=226&cmdCode=TOTAL&flowCode=M


,reporterDesc,partnerDesc,partner2Code,partner2Desc,refYear,flowDesc,primaryValue,cmdCode,qty,qtyUnitCode,qtyUnitAbbr,customsCode
0,China,Equatorial Guinea,0,World,2015,Import,"1,166,496,464.00",TOTAL,0.00,-1,NaN,C00
1,China,Equatorial Guinea,56,Belgium,2015,Import,"2,435.00",TOTAL,0.00,-1,NaN,C00
2,China,Equatorial Guinea,226,Equatorial Guinea,2015,Import,"1,166,493,970.00",TOTAL,0.00,-1,NaN,C00
3,China,Equatorial Guinea,344,China Hong Kong,2015,Import,59.00,TOTAL,0.00,-1,NaN,C00


A função `call_uncomtrade` passou a colocar `partner2Code = 0` quando não especificado,
para evitar o problema.

Neste exemplo não se inclui o parâmetro `partner2Code` e a função coloca a zero para obter o resultado
correcto.

In [33]:
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     reporterCode=m49_china,
                     partnerCode=partnerCode,
                     cmdCode='TOTAL',
                     period=period,
                     timeout=60,
                     echo_url=True
                     )
result = df.sort_values(['partnerDesc','flowCode'])[cols_partner2]
# print(result.to_markdown())
result

https://comtradeapi.un.org/public/v1/preview//C/A/HS?reporterCode=156&period=2015&partnerCode=226&partner2Code=0&cmdCode=TOTAL&flowCode=M&customsCode=C00


,reporterDesc,partnerDesc,partner2Code,partner2Desc,refYear,flowDesc,primaryValue,cmdCode,qty,qtyUnitCode,qtyUnitAbbr,customsCode
0,China,Equatorial Guinea,0,World,2015,Import,"1,166,496,464.00",TOTAL,0.00,-1,NaN,C00


#### Cobertura de informação sobre parter2

Não parece ser possível obter as informações referentes a partner2
 senão nos anos 2015-2017, quando o `reporter` é a China.

In [37]:
yrange = comtrade.year_range(2014,2023) 
reporterCode = m49_china


In [38]:

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     partner2Code=None,
                     cmdCode='TOTAL',
                     period=yrange,
                     timeout=10,
                     echo_url=True
                     )
result = df.sort_values(['partnerDesc','refYear','flowCode'])[cols_partner2]
# print(result.to_markdown())
result

https://comtradeapi.un.org/public/v1/preview//C/A/HS?reporterCode=156&period=2014%2C2015%2C2016%2C2017%2C2018%2C2019%2C2020%2C2021%2C2022&partnerCode=226&cmdCode=TOTAL&flowCode=M&customsCode=C00


,reporterDesc,partnerDesc,partner2Code,partner2Desc,refYear,flowDesc,primaryValue,cmdCode,qty,qtyUnitCode,qtyUnitAbbr,customsCode
0,China,Equatorial Guinea,0,World,2014,Import,"3,217,190,248.00",TOTAL,NaN,-1,NaN,C00
1,China,Equatorial Guinea,0,World,2015,Import,"1,166,496,464.00",TOTAL,0.00,-1,NaN,C00
2,China,Equatorial Guinea,56,Belgium,2015,Import,"2,435.00",TOTAL,0.00,-1,NaN,C00
3,China,Equatorial Guinea,226,Equatorial Guinea,2015,Import,"1,166,493,970.00",TOTAL,0.00,-1,NaN,C00
4,China,Equatorial Guinea,344,China Hong Kong,2015,Import,59.00,TOTAL,0.00,-1,NaN,C00
5,China,Equatorial Guinea,0,World,2016,Import,"631,851,506.00",TOTAL,0.00,-1,NaN,C00
6,China,Equatorial Guinea,24,Angola,2016,Import,"396,344.00",TOTAL,0.00,-1,NaN,C00
7,China,Equatorial Guinea,178,Congo,2016,Import,"1,457,849.00",TOTAL,0.00,-1,NaN,C00
8,China,Equatorial Guinea,226,Equatorial Guinea,2016,Import,"589,959,003.00",TOTAL,0.00,-1,NaN,C00
9,China,Equatorial Guinea,251,France and Monaco,2016,Import,"1,341.00",TOTAL,0.00,-1,NaN,C00


Mas outros países incluem esses dados em outros anos, por exemplo, Portugal

In [40]:
cmdCode = 'TOTAL'

period = "2018" ## if freqCode M  use aaaamm
flow = "X"
reporterCode=m49_portugal
partnerCode = m49_brazil
cmdCode='TOTAL'

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     partner2Code=None,
                     cmdCode=cmdCode,
                     customsCode=None,
                     period=period,
                     timeout=None,
                     echo_url=True
                     )
interesting_cols = [col for col in df.columns if len(df[col].unique())>1]
print(interesting_cols)
show_cols = cols_partner2 + list(set(interesting_cols)-set(cols_partner2))
result = df.sort_values(['partner2Code','flowCode'])[show_cols]

# print(result.to_markdown())
result

https://comtradeapi.un.org/public/v1/preview//C/A/HS?reporterCode=620&period=2018&partnerCode=76&cmdCode=TOTAL&flowCode=X
['partner2Code', 'partner2Desc', 'motCode', 'motDesc', 'isNetWgtEstimated', 'fobvalue', 'primaryValue', 'primaryValueFormated']


,reporterDesc,partnerDesc,partner2Code,partner2Desc,refYear,flowDesc,primaryValue,cmdCode,qty,qtyUnitCode,qtyUnitAbbr,customsCode,fobvalue,motCode,motDesc,isNetWgtEstimated,primaryValueFormated
0,Portugal,Brazil,0,World,2018,Export,"1,083,984,874.36",TOTAL,0.00,-1,NaN,C00,"1,083,984,874.36",0,All modes of transport,True,"1,083,984,874.361"
1,Portugal,Brazil,0,World,2018,Export,"52,669,113.05",TOTAL,0.00,-1,NaN,C00,"52,669,113.05",1000,Air,True,"52,669,113.046"
2,Portugal,Brazil,0,World,2018,Export,"1,027,354,167.57",TOTAL,0.00,-1,NaN,C00,"1,027,354,167.57",2100,Sea,True,"1,027,354,167.565"
3,Portugal,Brazil,0,World,2018,Export,"3,801,817.75",TOTAL,0.00,-1,NaN,C00,"3,801,817.75",3200,Road,True,"3,801,817.747"
4,Portugal,Brazil,0,World,2018,Export,"159,776.00",TOTAL,0.00,-1,NaN,C00,"159,776.00",9200,"Postal consignments, mail or courier shipment",True,"159,776.003"
5,Portugal,Brazil,251,France and Monaco,2018,Export,"56,538.91",TOTAL,0.00,-1,NaN,C00,"56,538.91",0,All modes of transport,False,"56,538.906"
6,Portugal,Brazil,251,France and Monaco,2018,Export,"56,538.91",TOTAL,0.00,-1,NaN,C00,"56,538.91",1000,Air,False,"56,538.906"
7,Portugal,Brazil,276,Germany,2018,Export,"18,143.93",TOTAL,0.00,-1,NaN,C00,"18,143.93",0,All modes of transport,False,"18,143.928"
8,Portugal,Brazil,276,Germany,2018,Export,"16,982.24",TOTAL,0.00,-1,NaN,C00,"16,982.24",1000,Air,False,"16,982.235"
9,Portugal,Brazil,276,Germany,2018,Export,"1,161.69",TOTAL,0.00,-1,NaN,C00,"1,161.69",2100,Sea,False,"1,161.694"


Note-se a duplicação de linhas para o mesmo terceto _reporterCode-partnerCode-partner2Code_

Essa duplicação deve-se a desdobramento `motCode` (_mode of transport_) e `partner2Code`. Se filtrarmos só por motCode == 0 
e partner2Code != 0 obtemos a lista de combinações reporter-partner-partner2 para todos os modos de tranporte, sem
valores agregados por partner.

In [41]:
result[(result.motCode==0)&(result.partner2Code != 0)][list(set(['refYear','reporterDesc','partnerDesc',]+interesting_cols))]

,fobvalue,refYear,primaryValue,motCode,motDesc,isNetWgtEstimated,reporterDesc,primaryValueFormated,partner2Desc,partnerDesc,partner2Code
5,"56,538.91",2018,"56,538.91",0,All modes of transport,False,Portugal,"56,538.906",France and Monaco,Brazil,251
7,"18,143.93",2018,"18,143.93",0,All modes of transport,False,Portugal,"18,143.928",Germany,Brazil,276
10,"235,517.71",2018,"235,517.71",0,All modes of transport,False,Portugal,"235,517.715",Greece,Brazil,300
12,"50,945.52",2018,"50,945.52",0,All modes of transport,False,Portugal,"50,945.523",Italy,Brazil,380
14,"15,883.18",2018,"15,883.18",0,All modes of transport,False,Portugal,"15,883.176",Netherlands,Brazil,528
16,"26,405.78",2018,"26,405.78",0,All modes of transport,False,Portugal,"26,405.78",Romania,Brazil,642
18,"16,145,881.59",2018,"16,145,881.59",0,All modes of transport,True,Portugal,"16,145,881.586",Spain,Brazil,724
21,"1,626.13",2018,"1,626.13",0,All modes of transport,False,Portugal,"1,626.135",United Kingdom,Brazil,826
23,"1,067,433,931.61",2018,"1,067,433,931.61",0,All modes of transport,True,Portugal,"1,067,433,931.612","Areas, not elsewhere specified",Brazil,899


Gravar em Excel (nome do ficheiro automaticamente reflecte o valor das variáveis relevantes)

In [42]:
result.to_excel(f"./downloads/partner2_{m49_reporter_codes_map[reporterCode]}_{m49_reporter_codes_map[partnerCode]}_{flow}_{period}.xlsx")

## Testes



In [45]:
import comtrade

period = "2016" ## if freqCode M  use aaaamm
flow = "M,X"
cmdCode = 'TOTAL'
reporterCode = m49_portugal
partnerCode = m49_angola

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '${:,.2f}'.format

cols_partner2 = ['reporterDesc','partnerDesc','partner2Code','partner2Desc','refYear',
        'flowCode','qtyUnitCode','primaryValue','isAggregate']
cols_partner2_qty_weight = ['reporterDesc','partnerDesc','partner2Code','partner2Desc','refYear','cmdDesc',
        'flowCode','primaryValue',
        'qtyUnitCode','qty','isQtyEstimated','altQtyUnitCode', 'altQtyUnitAbbr', 'altQty', 'isAltQtyEstimated',
       'netWgt', 'isNetWgtEstimated', 'grossWgt', 'isGrossWgtEstimated',
        'motCode']


df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     partner2Code=None,
                     cmdCode=cmdCode,
                     more_pars={'customsCode':None},
                     period=period,
                     timeout=30, echo_url=True
                     )

interesting_cols = [col for col in df.columns if len(df[col].unique())>1]
print(interesting_cols)
show_cols = cols_partner2 +  list(set(interesting_cols)-set(cols_partner2))
result = df.sort_values(['partnerDesc','flowCode'])[show_cols]
result

https://comtradeapi.un.org/public/v1/preview//C/A/HS?reporterCode=620&period=2016&partnerCode=24&cmdCode=TOTAL&flowCode=M%2CX
['flowCode', 'flowDesc', 'partner2Code', 'partner2Desc', 'motCode', 'motDesc', 'isNetWgtEstimated', 'cifvalue', 'fobvalue', 'primaryValue', 'primaryValueFormated']


,reporterDesc,partnerDesc,partner2Code,partner2Desc,refYear,flowCode,qtyUnitCode,primaryValue,isAggregate,fobvalue,cifvalue,motCode,motDesc,flowDesc,isNetWgtEstimated,primaryValueFormated
0,Portugal,Angola,0,World,2016,M,-1,"$895,881,332.31",True,NaN,"$895,881,332.31",0,All modes of transport,Import,True,"895,881,332.307"
2,Portugal,Angola,0,World,2016,M,-1,"$1,689,952.90",True,NaN,"$1,689,952.90",1000,Air,Import,True,"1,689,952.899"
4,Portugal,Angola,0,World,2016,M,-1,"$894,117,557.93",True,NaN,"$894,117,557.93",2100,Sea,Import,True,"894,117,557.927"
8,Portugal,Angola,0,World,2016,M,-1,"$1,080.88",True,NaN,"$1,080.88",9200,"Postal consignments, mail or courier shipment",Import,False,"1,080.876"
10,Portugal,Angola,0,World,2016,M,-1,"$72,740.61",True,NaN,"$72,740.61",9300,Self propelled goods,Import,False,"72,740.606"
12,Portugal,Angola,24,Angola,2016,M,-1,"$895,440,525.34",True,NaN,"$895,440,525.34",0,All modes of transport,Import,True,"895,440,525.343"
13,Portugal,Angola,24,Angola,2016,M,-1,"$1,689,952.90",True,NaN,"$1,689,952.90",1000,Air,Import,True,"1,689,952.899"
14,Portugal,Angola,24,Angola,2016,M,-1,"$893,676,750.96",True,NaN,"$893,676,750.96",2100,Sea,Import,True,"893,676,750.962"
15,Portugal,Angola,24,Angola,2016,M,-1,"$1,080.88",True,NaN,"$1,080.88",9200,"Postal consignments, mail or courier shipment",Import,False,"1,080.876"
16,Portugal,Angola,24,Angola,2016,M,-1,"$72,740.61",True,NaN,"$72,740.61",9300,Self propelled goods,Import,False,"72,740.606"
